In [18]:
import os
import glob
import pandas as pd
import AMD_Tools3 as amd
%matplotlib inline

In [3]:
simdata = [pd.read_csv(csvpath) for csvpath in glob.glob('simdata/*.csv')]
simdata = pd.concat(simdata, axis=0)

In [21]:
import folium
from folium.plugins import HeatMap

simdata_grouped = simdata.groupby(by=['meshcode', 'year']).DVI.max()
print('len(DVI<1.9):', (simdata_grouped < 1.9).sum())
print('len(simdata):', len(simdata_grouped))

def meshcode2lat(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[0]

def meshcode2lon(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[1]

simdata = simdata_grouped.reset_index()
simdata['lat'] = simdata.meshcode.astype(str).apply(meshcode2lat)
simdata['lon'] = simdata.meshcode.astype(str).apply(meshcode2lon)

for year in simdata.year.unique():
    map = folium.Map([35.962500, 139.11875], tiles='stamentoner', zoom_start=8)
    df = simdata[simdata.year == year]
    df = df[['lat', 'lon', 'DVI']].copy()
    df.dropna(how='any', inplace=True)
    data = df.as_matrix().tolist()
    HeatMap(data).add_to(map)
    map.save(os.path.join('output', f'heatmap_{year}.html'))
map

len(DVI<1.9): 5406
len(simdata): 11803
